# Using Agent Framework in Jupyter Notebook

This notebook demonstrates how to use the agent_framework package with structured output.


In [1]:
# Import the framework
from agent_framework import Agent, LlmClient
from pydantic import BaseModel
from typing import Literal, List


In [2]:
# Define your structured output model
class SentimentAnalysis(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"]
    confidence: float
    key_phrases: List[str]


In [3]:
# Create agent with structured output
agent = Agent(
    model=LlmClient(model="gpt-5-mini"),
    tools=[],
    instructions="Analyze the sentiment of the provided text.",
    output_type=SentimentAnalysis
)


In [4]:
# Run the agent
result = await agent.run("This product exceeded my expectations! Highly recommend.")

# Access structured output
print(f"Sentiment: {result.output.sentiment}")      # "positive"
print(f"Confidence: {result.output.confidence}")    # 0.92
print(f"Key phrases: {result.output.key_phrases}")  # ["exceeded expectations", "highly recommend"]


Sentiment: positive
Confidence: 0.95
Key phrases: ['exceeded my expectations', 'Highly recommend', 'product']


In [ ]:
'''
To implement agent, we need tools, execution context, 
instructions(system prompt that defines agent behavior) and an llm. 

Event: It is a record of who did what ? like was it user request, 
or llm requested tool call, or did we get a result back from the tool etc., 

'''

from anyio import Event
from agent_framework import ExecutionContext, Message
from agent_framework.agent import AgentResult

class Agent: ## does this inherit from anything ? 
    def init(self, tools, executionContext, llmClient, instructions, maxSteps, verbose, name = "agent"):
        self.tools = self._setup_tools(tools or [])
        self.executionContext = executionContext
        self.llmClient = llmClient
        self.instructions = instructions
        self.maxSteps = maxSteps
        self.verbose = verbose
        self.name = name

    ## step 1 is to setup tools

    def _setup_tools(self, tools):
        return tools

    ## step 2 is to define entry point for users.(run method)

    async def run(self, user_input, context):

        ## check if there is any previous context, else create

        if context is None:
            context = ExecutionContext()

        ## add the user_event to the event
        user_event = Event(
            execution_id = context.execution_id,
            author = 'user',
            content = [Message(role = 'user', content = user_input)]
        )
        ## add the event to context
        context.add_event(user_event)

        ## if agent doesnt reach final result or max steps, keep performing
        while not context.final_result and context.current_step < self.max_steps:
            ## each step is a think-act cycle
            await self.step(context)

            ## check if newly performed action is final
            last_event = context.events[-1]

            # If it is final, then extract the last event and sent it to 
            # Agent result along with the context
            if self._is_final_response(last_event):
                context.final_result = self._extract_final_result(last_event)

        return AgentResult(context.final_result, context = context)

    # step 3 prepare for llm request

    def _prepare_llm_request(self, context):
        
        #flatten all the events (why ?)
        flat_contents = []
        for event in context.events:
            flat_contents.extend(event.content)

        ## with this context, call llm
        return LlmRequest(
        instructions=[self.instructions] if self.instructions else [],
        contents=flat_contents,
        tools=self.tools,
        tool_choice="auto" if self.tools else None,
    )

    async def step(self, context):
        
        ## write a method for this
        llm_request = self._prepare_llm_request(context)

        # Get LLM's decision
        llm_response = await self.think(llm_request)

        response_event = Event(
            execution_id=context.execution_id,
            author=self.name,
            content=llm_response.content,
        )

    async def think(self, llm_request):
        """Get LLM's response/decision."""
        return await self.model.generate(llm_request)













Q. LLM Request? 

A. It goes from our agent to LLM call. before sending it, we bundle it with necessary context , prompt and tools.